In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:
import streamlit as st
import pandas as pd
import base64
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from PIL import Image

import warnings
warnings.filterwarnings('ignore')


from yahooquery import Ticker
from fbprophet import Prophet
import yfinance as yf
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense,LSTM
#import stocker
 
import datetime as dt 
dia = dt.datetime.today().strftime(format='20%y-%m-%d')



import re
import urllib.request
import urllib.parse
import http.cookiejar
import time
import lxml

from lxml.html import fragment_fromstring
from collections import OrderedDict
import json
import ast
import datetime
import os
from pymongo import MongoClient

from datetime import datetime



# ------------------------------ SCRAPPING -------------------------------


def get_data(*args, **kwargs):
    url = 'http://www.fundamentus.com.br/resultado.php'
    cj = http.cookiejar.CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201'),
                         ('Accept', 'text/html, text/plain, text/css, text/sgml, */*;q=0.01')]

    # Aqui estão os parâmetros de busca das ações
    # Estão em branco para que retorne todas as disponíveis
    data = {'pl_min':'','pl_max':'','pvp_min':'','pvp_max' :'','psr_min':'','psr_max':'','divy_min':'','divy_max':'',            'pativos_min':'','pativos_max':'','pcapgiro_min':'','pcapgiro_max':'','pebit_min':'','pebit_max':'', 'fgrah_min':'',
            'fgrah_max':'', 'firma_ebit_min':'', 'firma_ebit_max':'','margemebit_min':'','margemebit_max':'',            'margemliq_min':'','margemliq_max':'', 'liqcorr_min':'','liqcorr_max':'','roic_min':'','roic_max':'','roe_min':'',            'roe_max':'','liq_min':'','liq_max':'','patrim_min':'','patrim_max':'','divbruta_min':'','divbruta_max':'',         'tx_cresc_rec_min':'','tx_cresc_rec_max':'','setor':'','negociada':'ON','ordem':'1','x':'28','y':'16'}

    with opener.open(url, urllib.parse.urlencode(data).encode('UTF-8')) as link:
        content = link.read().decode('ISO-8859-1')

    pattern = re.compile('<table id="resultado".*</table>', re.DOTALL)
    reg = re.findall(pattern, content)[0]
    page = fragment_fromstring(reg)
    lista = OrderedDict()

    stocks = page.xpath('tbody')[0].findall("tr")

    todos = []
    for i in range(0, len(stocks)):
        lista[i] = {
            stocks[i].getchildren()[0][0].getchildren()[0].text: {
                'cotacao': stocks[i].getchildren()[1].text,
               'P/L': stocks[i].getchildren()[2].text,
               'P/VP': stocks[i].getchildren()[3].text,
               'PSR': stocks[i].getchildren()[4].text,
               'DY': stocks[i].getchildren()[5].text,
               'P/Ativo': stocks[i].getchildren()[6].text,
               'P/Cap.Giro': stocks[i].getchildren()[7].text,
               'P/EBIT': stocks[i].getchildren()[8].text,
               'P/Ativ.Circ.Liq.': stocks[i].getchildren()[9].text,
               'EV/EBIT': stocks[i].getchildren()[10].text,
               'EBITDA': stocks[i].getchildren()[11].text,
               'Mrg. Ebit': stocks[i].getchildren()[12].text,
               'Mrg.Liq.': stocks[i].getchildren()[13].text,
               'Liq.Corr.': stocks[i].getchildren()[14].text,
               'ROIC': stocks[i].getchildren()[15].text,
               'ROE': stocks[i].getchildren()[16].text,
               'Liq.2m.': stocks[i].getchildren()[17].text,
               'Pat.Liq': stocks[i].getchildren()[18].text,
               'Div.Brut/Pat.': stocks[i].getchildren()[19].text,
               'Cresc.5a': stocks[i].getchildren()[20].text
               }
            }

    return lista

def get_specific_data(stock):
    url = "http://www.fundamentus.com.br/detalhes.php?papel=" + stock
    cj = http.cookiejar.CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201'),
                         ('Accept', 'text/html, text/plain, text/css, text/sgml, */*;q=0.01')]
    
    # Get data from site
    link = opener.open(url, urllib.parse.urlencode({}).encode('UTF-8'))
    content = link.read().decode('ISO-8859-1')

    # Get all table instances
    pattern = re.compile('<table class="w728">.*</table>', re.DOTALL)
    reg = re.findall(pattern, content)[0]
    reg = "<div>" + reg + "</div>"
    page = fragment_fromstring(reg)
    all_data = {}

    # There is 5 tables with tr, I will get all trs
    all_trs = []
    all_tables = page.xpath("table")

    for i in range(0, len(all_tables)):
        all_trs = all_trs + all_tables[i].findall("tr")

    # Run through all the trs and get the label and the
    # data for each line
    for tr_index in range(0, len(all_trs)):
        tr = all_trs[tr_index]
        # Get into td
        all_tds = tr.getchildren()
        for td_index in range(0, len(all_tds)):
            td = all_tds[td_index]

            label = ""
            data = ""

            # The page has tds with contents and some 
            # other with not
            if (td.get("class").find("label") != -1):
                # We have a label
                for span in td.getchildren():
                    if (span.get("class").find("txt") != -1):
                        label = span.text

                # If we did find a label we have to look 
                # for a value 
                if (label and len(label) > 0):
                    next_td = all_tds[td_index + 1]

                    if (next_td.get("class").find("data") != -1):
                        # We have a data
                        for span in next_td.getchildren():
                            if (span.get("class").find("txt") != -1):
                                if (span.text):
                                    data = span.text
                                else:
                                    # If it is a link
                                    span_children = span.getchildren()
                                    if (span_children and len(span_children) > 0):
                                        data = span_children[0].text

                                # Include into dict
                                all_data[label] = data

                                # Erase it
                                label = ""
                                data = ""

    return all_data


def flatten(d):
    '''
    Flatten an OrderedDict object
    '''
    result = OrderedDict()
    for k, v in d.items():
        if isinstance(v, dict):
            result.update(flatten(v))
        else:
            result[k] = v
    return result

# ----------------------------------SIDEBAR -------------------------------------------------------------
def main():

    st.sidebar.header("Explorador de ativos")
    n_sprites = st.sidebar.radio(
        "Escolha uma opção", options=["Análise técnica e fundamentalista", "Comparação de ativos","Descobrir novos ativos", "Rastreador de trade", "Análise de carteira e previsão de lucro"], index=0
    )

    st.sidebar.markdown('É preciso ter paciência e disciplina para se manter firme em suas convicções quando o mercado insiste que você está errado.!')
    st.sidebar.markdown('Benjamin Graham')
    st.sidebar.markdown('Email para contato: lucas.vasconcelos3@gmail.com')
    st.sidebar.markdown('Portfólio: https://github.com/lucasvascrocha')                                    

# ------------------------------ INÍCIO ANÁLISE TÉCNICA E FUNDAMENTALISTA ----------------------------             

    if n_sprites == "Análise técnica e fundamentalista":
        st.image('https://media.giphy.com/media/rM0wxzvwsv5g4/giphy.gif', width=400)    
        #image = Image.open('imagens/logo.jpg')
        #st.image(image, use_column_width=True)                       
        st.title('Análise Técnica e fundamentalista')
        st.subheader('Escolha o ativo que deseja analisar e pressione enter')
        nome_do_ativo = st.text_input('Nome do ativo')


        st.write('Este explorador funciona melhor para ações, porém também suporta alguns fundos imobiliários')    
        st.write('Os parâmetros utilizados em grande maioria foram seguindo as teorias de Benjamin Graham')

        if nome_do_ativo != "":
            nome_do_ativo = str(nome_do_ativo + '.SA')
            st.subheader('Analisando os dados')
            df = Ticker(nome_do_ativo,country='Brazil')
            time = df.history( period='max')
            st.dataframe(time.tail())

# ------------------------------ RESUMO ---------------------------- 

            resumo = pd.DataFrame(df.summary_detail)
            resumo = resumo.transpose()
            if len(nome_do_ativo) == 8:
              fundamentus = get_specific_data(nome_do_ativo[:5])
              fundamentus = pd.DataFrame([fundamentus])
              
              try:
                    
                  pfizer = yf.Ticker(nome_do_ativo)
                  info = pfizer.info 
                  st.title('PERFIL DA EMPRESA')
                  st.subheader(info['longName']) 
                  st.markdown('** Setor **: ' + info['sector'])
                  st.markdown('** Atividade **: ' + info['industry'])
                  st.markdown('** Website **: ' + info['website'])
              except:
                exit
                
              try:
                fundInfo = {
            'Dividend Yield (%) -12 meses': round(info['dividendYield']*100,2),
            'P/L': fundamentus['P/L'][0],
            'P/VP': fundamentus['P/VP'][0],
            'Próximo pagamento de dividendo:': (pfizer.calendar.transpose()['Earnings Date'].dt.strftime('%d/%m/%Y')[0])
        }   
                fundDF = pd.DataFrame.from_dict(fundInfo, orient='index')
                fundDF = fundDF.rename(columns={0: 'Valores'})
                st.subheader('Informações fundamentalistas') 
                st.table(fundDF)
              except:
                exit
              
            else:
              st.write('---------------------------------------------------------------------')
              st.dataframe(resumo) 
              pfizer = yf.Ticker(nome_do_ativo)
              info = pfizer.info 
              st.title('Company Profile')
              st.subheader(info['longName']) 
              try:
                st.markdown('** Sector **: ' + info['sector'])
                st.markdown('** Industry **: ' + info['industry'])
                st.markdown('** Website **: ' + info['website'])
              except:
                exit
            
# ------------------------------ GRÁFICOS DE RENDIMENTO ---------------------------- 

            if len(nome_do_ativo) == 8:
              
              import datetime
              fundamentalist = df.income_statement()
              fundamentalist['data'] = fundamentalist['asOfDate'].dt.strftime('%d/%m/%Y')
              fundamentalist = fundamentalist.drop_duplicates('asOfDate')
              fundamentalist = fundamentalist.loc[fundamentalist['periodType'] == '12M']

              #volatilidade
              TRADING_DAYS = 360
              returns = np.log(time['close']/time['close'].shift(1))
              returns.fillna(0, inplace=True)
              volatility = returns.rolling(window=TRADING_DAYS).std()*np.sqrt(TRADING_DAYS)
              vol = pd.DataFrame(volatility.iloc[-360:]).reset_index()

              #sharpe ratio
              sharpe_ratio = returns.mean()/volatility
              sharpe = pd.DataFrame(sharpe_ratio.iloc[-360:]).reset_index()

              div = time.reset_index()
              div['year'] = pd.to_datetime(div['date']).dt.strftime('%Y')
              div_group = div.groupby('year').agg({'close':'mean','dividends':'sum'})
              div_group['dividendo(%)'] = round((div_group['dividends'] * 100 ) / div_group['close'],4)

              from plotly.subplots import make_subplots
              fig = make_subplots(
                  rows=3, cols=2,
                  specs=[[{"type": "bar"}, {"type": "bar"}],
                        [{"type": "bar"}, {"type": "bar"}],
                        [{"type": "scatter"}, {"type": "scatter"}]],
                    subplot_titles=("Receita Total","Lucro",'Dividendos (%)','Dividendos unitário R$','Volatilidade', 'Sharpe ratio (Retorno/ Risco)')
              )

              fig.add_trace(go.Bar(x =pfizer.financials.transpose().index,  y=pfizer.financials.transpose()['Total Revenue']), row=1, col=1)

              fig.add_trace(go.Bar(x =pfizer.financials.transpose().index,  y=pfizer.financials.transpose()['Net Income From Continuing Ops']), row=1, col=2)

              fig.add_trace(go.Bar(x =div_group.reset_index().tail(5)['year'],  y=div_group.reset_index().tail(5)['dividendo(%)']),row=2, col=1)

              fig.add_trace(go.Bar(x =div_group.reset_index().tail(5)['year'],  y=div_group.reset_index().tail(5)['dividends']),row=2, col=2)

              fig.add_trace(go.Scatter(x =vol['date'],  y=vol['close']),row=3, col=1)

              fig.add_trace(go.Scatter(x =sharpe['date'],  y=sharpe['close']),row=3, col=2)

              fig.update_layout(height=800, showlegend=False)

              st.plotly_chart(fig)

            else:
                #volatilidade
              TRADING_DAYS = 160
              returns = np.log(time['close']/time['close'].shift(1))
              returns.fillna(0, inplace=True)
              volatility = returns.rolling(window=TRADING_DAYS).std()*np.sqrt(TRADING_DAYS)
              vol = pd.DataFrame(volatility.iloc[-160:]).reset_index()

              #sharpe ratio
              sharpe_ratio = returns.mean()/volatility
              sharpe = pd.DataFrame(sharpe_ratio.iloc[-160:]).reset_index()

              from plotly.subplots import make_subplots
              fig = make_subplots(
                  rows=1, cols=2,
                  specs=[[{"type": "scatter"}, {"type": "scatter"}]],
                    subplot_titles=('Volatilidade', 'Sharpe ratio (Retorno/ Risco)')
              )

              fig.add_trace(go.Scatter(x =vol['date'],  y=vol['close']),row=1, col=1)

              fig.add_trace(go.Scatter(x =sharpe['date'],  y=sharpe['close']),row=1, col=2)

              fig.update_layout(height=800, showlegend=False)

              st.plotly_chart(fig)

# ------------------------------ GRÁFICOS DE Candlestick---------------------------- 
            
            fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'), 
               row_width=[0.2, 0.7])

            # Plot OHLC on 1st row
            fig.add_trace(go.Candlestick(x=time.reset_index()['date'][-90:],
                            open=time['open'][-90:], high=time['high'][-90:],
                            low=time['low'][-90:], close=time['close'][-90:], name="OHLC"), 
                            row=1, col=1)            

            # Bar trace for volumes on 2nd row without legend
            fig.add_trace(go.Bar(x=time.reset_index()['date'][-90:], y=time['volume'][-90:], showlegend=False), row=2, col=1)

            # Do not show OHLC's rangeslider plot 
            fig.update(layout_xaxis_rangeslider_visible=False)
            fig.update_layout(autosize=False,width=800,height=800,)
            st.plotly_chart(fig)
            
# ------------------------------ GRÁFICOS DE Retorno acumulado---------------------------- 

            layout = go.Layout(title="Retorno acumulado",xaxis=dict(title="Data"), yaxis=dict(title="Retorno"))
            fig = go.Figure(layout = layout)
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-365:], y=time.reset_index()['close'][-365:].pct_change().cumsum(), mode='lines', line_width=3,line_color='rgb(0,0,0)'))
            fig.update_layout(autosize=False,width=800,height=800,)

            st.plotly_chart(fig)

# ------------------------------ GRÁFICOS DE Médias móveis---------------------------- 

            rolling_50  = time['close'].rolling(window=50)
            rolling_mean_50 = rolling_50.mean()

            rolling_20  = time['close'].rolling(window=20)
            rolling_mean_20 = rolling_20.mean()

            rolling_10  = time['close'].rolling(window=10)
            rolling_mean_10 = rolling_10.mean()

            layout = go.Layout(title="Médias móveis",xaxis=dict(title="Data"), yaxis=dict(title="Preço R$"))
            fig = go.Figure(layout = layout)
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-120:], y=time["close"][-120:], mode='lines', line_width=3,name='Real',line_color='rgb(0,0,0)'))
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-120:], y=rolling_mean_50[-120:],mode='lines',name='MM(50)',opacity = 0.6))
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-120:], y=rolling_mean_20[-120:],mode='lines',name='MM(20)',opacity = 0.6))
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-120:], y=rolling_mean_10[-120:],mode='lines',name='MM(10)',opacity = 0.6,line_color='rgb(100,149,237)'))
            # fig.add_trace(go.Candlestick(x=time.reset_index()['date'][-120:], open=time['open'][-120:],high=time['high'][-120:],low=time['low'][-120:],close=time['close'][-120:]))
            fig.update_layout(autosize=False,width=800,height=800,)

            st.plotly_chart(fig)

# ------------------------------ GRÁFICOS DE Retração de Fibonacci---------------------------- 

            time_fibo = time.copy()
            periodo_fibonacci = int(st.number_input(label='periodo fibonacci',value=90))
            
            Price_Min =time_fibo[-periodo_fibonacci:]['low'].min()
            Price_Max =time_fibo[-periodo_fibonacci:]['high'].max()

            Diff = Price_Max-Price_Min
            level1 = Price_Max - 0.236 * Diff
            level2 = Price_Max - 0.382 * Diff
            level3 = Price_Max - 0.618 * Diff
         
            st.write ('0% >>' f'{round(Price_Max,2)}')
            st.write ('23,6% >>' f'{round(level1,2)}')
            st.write ('38,2% >>' f'{round(level2,2)}')
            st.write ('61,8% >>' f'{round(level3,2)}')
            st.write ('100% >>' f'{round(Price_Min,2)}')

            time_fibo['Price_Min'] = Price_Min
            time_fibo['level1'] = level1
            time_fibo['level2'] = level2
            time_fibo['level3'] = level3
            time_fibo['Price_Max'] = Price_Max

            layout = go.Layout(title=f'Retração de Fibonacci',xaxis=dict(title="Data"), yaxis=dict(title="Preço"))
            fig = go.Figure(layout = layout)
            fig.add_trace(go.Scatter(x=time_fibo[-periodo_fibonacci:].reset_index()['date'], y=time_fibo[-periodo_fibonacci:].close, mode='lines', line_width=3,name='Preço real',line_color='rgb(0,0,0)'))
            fig.add_trace(go.Scatter(x=time_fibo[-periodo_fibonacci:].reset_index()['date'], y=time_fibo[-periodo_fibonacci:].Price_Min, mode='lines', line_width=0.5,name='100%',line_color='rgb(255,0,0)',))
            fig.add_trace(go.Scatter(x=time_fibo[-periodo_fibonacci:].reset_index()['date'], y=time_fibo[-periodo_fibonacci:].level3, mode='lines', line_width=0.5,name='61,8%',line_color='rgb(255,255,0)',fill= 'tonexty', fillcolor ="rgba(255, 0, 0, 0.2)"))
            fig.add_trace(go.Scatter(x=time_fibo[-periodo_fibonacci:].reset_index()['date'], y=time_fibo[-periodo_fibonacci:].level2, mode='lines', line_width=0.5,name='38,2%',line_color='rgb(0,128,0)',fill= 'tonexty', fillcolor ="rgba(255, 255, 0, 0.2)"))
            fig.add_trace(go.Scatter(x=time_fibo[-periodo_fibonacci:].reset_index()['date'], y=time_fibo[-periodo_fibonacci:].level1, mode='lines', line_width=0.5,name='23,6%',line_color='rgb(128,128,128)',fill= 'tonexty', fillcolor ="rgba(0, 128, 0, 0.2)"))
            fig.add_trace(go.Scatter(x=time_fibo[-periodo_fibonacci:].reset_index()['date'], y=time_fibo[-periodo_fibonacci:].Price_Max, mode='lines', line_width=0.5,name='0%',line_color='rgb(0,0,255)',fill= 'tonexty', fillcolor ="rgba(128, 128, 128, 0.2)"))
            fig.update_layout(autosize=False,width=800,height=800,)

            st.plotly_chart(fig)

# ------------------------------ GRÁFICOS DE RSI---------------------------- 

            periodo_RSI = int(st.number_input(label='periodo RSI',value=90))

            delta = time['close'][-periodo_RSI:].diff()
            up, down = delta.copy(), delta.copy()

            up[up < 0] = 0
            down[down > 0] = 0

            period = 14
                
            rUp = up.ewm(com=period - 1,  adjust=False).mean()
            rDown = down.ewm(com=period - 1, adjust=False).mean().abs()

            time['RSI_' + str(period)] = 100 - 100 / (1 + rUp / rDown)
            time['RSI_' + str(period)].fillna(0, inplace=True)

            layout = go.Layout(title=f'RSI {periodo_RSI}',xaxis=dict(title="Data"), yaxis=dict(title="%RSI"))
            fig = go.Figure(layout = layout)
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-periodo_RSI:], y=round(time['RSI_14'][-periodo_RSI:],2), mode='lines', line_width=3,name=f'RSI {periodo_RSI}',line_color='rgb(0,0,0)'))

            fig.update_layout(autosize=False,width=800,height=800,)

            st.plotly_chart(fig)

# ------------------------------ GRÁFICOS DE pivôs---------------------------- 

            periodo_pivo = int(st.number_input(label='periodo pivô',value=20))

            time['PP'] = pd.Series((time['high'] + time['low'] + time['close']) /3)  
            time['R1'] = pd.Series(2 * time['PP'] - time['low'])  
            time['S1'] = pd.Series(2 * time['PP'] - time['high'])  
            time['R2'] = pd.Series(time['PP'] + time['high'] - time['low'])  
            time['S2'] = pd.Series(time['PP'] - time['high'] + time['low']) 

            layout = go.Layout(title=f'Pivô',xaxis=dict(title="Data"), yaxis=dict(title="Preço"))
            fig = go.Figure(layout = layout)
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-periodo_pivo:], y=round(time['close'][-periodo_pivo:],2), mode='lines', line_width=3,name=f'preço real',line_color='rgb(0,0,0)'))
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-periodo_pivo:], y=round(time['PP'][-periodo_pivo:],2), mode='lines', line_width=1,name=f'Ponto do pivô',line_color='rgb(0,128,0)'))
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-periodo_pivo:], y=round(time['R1'][-periodo_pivo:],2), mode='lines', line_width=1,name=f'Resistência 1',line_color='rgb(100,149,237)'))
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-periodo_pivo:], y=round(time['S1'][-periodo_pivo:],2), mode='lines', line_width=1,name=f'Suporte 1',line_color='rgb(100,149,237)'))
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-periodo_pivo:], y=round(time['R2'][-periodo_pivo:],2), mode='lines', line_width=1,name=f'Resistência 2',line_color='rgb(255,0,0)'))
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-periodo_pivo:], y=round(time['S2'][-periodo_pivo:],2), mode='lines', line_width=1,name=f'Suporte 2',line_color='rgb(255,0,0)'))
            fig.update_layout(autosize=False,width=800,height=800,)

            st.plotly_chart(fig)

# ------------------------------ GRÁFICOS DE Bolinger---------------------------- 

            periodo_bolinger = int(st.number_input(label='periodo Bolinger',value=180))

            time['MA20'] = time['close'].rolling(20).mean()
            time['20 Day STD'] = time['close'].rolling(window=20).std()
            time['Upper Band'] = time['MA20'] + (time['20 Day STD'] * 2)
            time['Lower Band'] = time['MA20'] - (time['20 Day STD'] * 2)

            layout = go.Layout(title=f'Banda de Bolinger',xaxis=dict(title="Data"), yaxis=dict(title="Preço"))
            fig = go.Figure(layout = layout)
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-periodo_bolinger:], y=round(time['Upper Band'][-periodo_bolinger:],2), mode='lines', line_width=1,name=f'Banda superior',line_color='rgb(255,0,0)'))
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-periodo_bolinger:], y=round(time['Lower Band'][-periodo_bolinger:],2), mode='lines', line_width=1,name=f'Banda inferior',line_color='rgb(255,0,0)',fill= 'tonexty', fillcolor ="rgba(255, 0, 0, 0.1)",opacity=0.2))
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-periodo_bolinger:], y=round(time['close'][-periodo_bolinger:],2), mode='lines', line_width=3,name=f'preço real',line_color='rgb(0,0,0)'))
            fig.add_trace(go.Scatter(x=time.reset_index()['date'][-periodo_bolinger:], y=round(time['MA20'][-periodo_bolinger:],2), mode='lines', line_width=2,name=f'MM 20',line_color='rgb(0,128,0)'))
            fig.update_layout(autosize=False,width=800,height=800,)

            st.plotly_chart(fig)

# ------------------------------ Previsões---------------------------- 

            st.subheader('Previsões')

            st.write('As previsões são feitas levando em conta apenas o movimento gráfico, porém o movimento do preço de um ativo é influenciado por diversos outros fatores, com isso, deve se considerar as previsões como uma hipótese de o preço do ativo variar somente pela sua variação gráfica')

            st.write('Previsão considerando os últimos 365 dias, pode ser entendida como uma tendência dos dados segundo o último ano')

            time = time.reset_index()
            time = time[['date','close']]
            time.columns = ['ds','y']

            #Modelling
            m = Prophet()
            m.fit(time[-360:])
            future = m.make_future_dataframe(periods=30)
            forecast = m.predict(future[-30:])

            from fbprophet.plot import plot_plotly, plot_components_plotly

            fig1 = plot_plotly(m, forecast)
            st.plotly_chart(fig1)
            #st.plotly_chart(m, forecast)
            fig2 = m.plot_components(forecast)
            st.plotly_chart(fig2)

            #st.write('Previsão considerando as últimas semanas, pode ser entendida como uma tendência dos dados segundo os últimos dias. Leva em consideração diversos fatores como: Índice de força relativa RSI, oscilador estocástico %K, Indicador Willian %R além do movimento gráfico dos últimos dias')

            #predict = stocker.predict.tomorrow(nome_do_ativo)

            #st.write('Previsão para o dia:',f'{predict[2]}','é que a ação feche no valor de: R$',f'{predict[0]}')

            #preço_ontem= round(time['y'][-1:].values[0],2)
            #if predict[0] < preço_ontem:
                #st.write('Previsão para o dia:',f'{predict[2]}','é que a ação caia de ',f'{preço_ontem}', 'para valor de: R$ ',f'{predict[0]}')
            #else:
                #st.write('Previsão para o dia:',f'{predict[2]}','é que a ação suba de ',f'{preço_ontem}', 'para valor de: R$ ',f'{predict[0]}')
                         
# ------------------------------ INÍCIO Comparação de ativos ------------------------------------------------------------------------------------

    if n_sprites == "Comparação de ativos":

        st.image('https://media.giphy.com/media/JtBZm3Getg3dqxK0zP/giphy.gif', width=300)    
        #image = Image.open('imagens/logo.jpg')
        #st.image(image, use_column_width=True)                       
        st.title('Comparação de ativos')
        st.subheader('Escolha até 4 ativos para comparar')
        nome_do_ativo1 = st.text_input('Nome do 1º ativo')
        nome_do_ativo2 = st.text_input('Nome do 2º ativo')
        nome_do_ativo3 = st.text_input('Nome do 3º ativo')
        nome_do_ativo4 = st.text_input('Nome do 4º ativo')
        
        if nome_do_ativo4 != "":
            st.subheader('Analisando os dados')
            nome_do_ativo1 = str(nome_do_ativo1 + '.SA')
            nome_do_ativo2 = str(nome_do_ativo2 + '.SA')
            nome_do_ativo3 = str(nome_do_ativo3 + '.SA')
            nome_do_ativo4 = str(nome_do_ativo4 + '.SA')
            
            df = Ticker([nome_do_ativo1,nome_do_ativo2,nome_do_ativo3,nome_do_ativo4],country='Brazil')
            time = df.history( start='2018-01-01', end = (dt.datetime.today() + dt.timedelta(days=1)).strftime(format='20%y-%m-%d'))
            lista = get_data()
            todos = pd.DataFrame(flatten(lista).keys()).transpose()
            todos.columns = todos.iloc[0]

            for i in range(len(lista)):
              todos = pd.concat([todos,pd.DataFrame(lista[i]).transpose()])

            todos = todos.iloc[1:]
            todos['P/L'] = todos['P/L'].str.replace('.','')
            todos['DY'] = todos['DY'].str.replace('%','')
            todos['Liq.2m.'] = todos['Liq.2m.'].str.replace('.','')
            todos['Pat.Liq'] = todos['Pat.Liq'].str.replace('.','')
            todos = todos.replace(',','.', regex=True)
            todos = todos.apply(pd.to_numeric,errors='ignore')

            comparar = todos.loc[todos.index.isin([nome_do_ativo1[:5],nome_do_ativo2[:5],nome_do_ativo3[:5],nome_do_ativo4[:5]])]
            
            st.dataframe(comparar)

# ------------------------------ INÍCIO Comparação DY ---------------
            
            layout = go.Layout(title="DY",xaxis=dict(title="Ativo"), yaxis=dict(title="DY %"))
            fig = go.Figure(layout = layout)
            fig.add_trace(go.Bar(x=comparar.sort_values('DY',ascending=True).index, y=comparar.sort_values('DY',ascending=True)['DY'] ))

            fig.update_layout(autosize=False,width=800,height=400,)

            st.plotly_chart(fig)

# ------------------------------ INÍCIO Comparação P/L ---------------

            layout = go.Layout(title="P/L",xaxis=dict(title="Ativo"), yaxis=dict(title="P/L"))
            fig = go.Figure(layout = layout)
            fig.add_trace(go.Bar(x=comparar.sort_values('P/L',ascending=True).index, y=comparar.sort_values('P/L',ascending=True)['P/L'] ))

            fig.update_layout(autosize=False,width=800,height=400,)

            st.plotly_chart(fig)

# ------------------------------ INÍCIO Comparação P/V---------------

            layout = go.Layout(title="P/VP",xaxis=dict(title="Ativo"), yaxis=dict(title="P/VP"))
            fig = go.Figure(layout = layout)
            fig.add_trace(go.Bar(x=comparar.sort_values('P/VP',ascending=True).index, y=comparar.sort_values('P/VP',ascending=True)['P/VP'] ))

            fig.update_layout(autosize=False,width=800,height=400,)

            st.plotly_chart(fig)

# ------------------------------ INÍCIO Comparação P/L * P/VP---------------

            layout = go.Layout(title="P/L X P/VP",xaxis=dict(title="Ativo"), yaxis=dict(title="P/L X P/VP"))
            fig = go.Figure(layout = layout)
            fig.add_trace(go.Bar(x=comparar.index, y=comparar['P/L'] * comparar['P/VP'] ))

            fig.update_layout(autosize=False,width=800,height=400,)

            st.plotly_chart(fig)

# ------------------------------ GRÁFICOS DE retorno acumulado---------------------------- 

            periodo_inicio = int(st.number_input(label='periodo retorno acumulado',value=360))

            ret = time.reset_index()
            layout = go.Layout(title="Retorno acumulado",xaxis=dict(title="Data"), yaxis=dict(title="Retorno"))
            fig = go.Figure(layout = layout)
            for i in range(len(ret['symbol'].unique())):
              fig.add_trace(go.Scatter(x=ret.loc[ret['symbol']==ret['symbol'].unique()[i]][-periodo_inicio:]['date'], y=ret.loc[ret['symbol']==ret['symbol'].unique()[i]][-periodo_inicio:]['close'].pct_change().cumsum(),mode='lines',name=ret.reset_index()['symbol'].unique()[i]))


            fig.update_layout(autosize=False,width=800,height=800,)

            st.plotly_chart(fig)

# ------------------------------ GRÁFICOS DE MÉDIAS MÓVEIS 50---------------------------- 

            rolling_50  = time['close'].rolling(window=50)
            rolling_mean_50 = rolling_50.mean()
            rolling_mean_50 = pd.DataFrame(rolling_mean_50.reset_index())
            # mm50 = time.reset_index()


            layout = go.Layout(title="MÉDIAS MÓVEIS 50",xaxis=dict(title="Data"), yaxis=dict(title="Preço R$"))
            fig = go.Figure(layout = layout)
            for i in range(len(rolling_mean_50['symbol'].unique())):
              fig.add_trace(go.Scatter(x=rolling_mean_50.loc[rolling_mean_50['symbol']==rolling_mean_50['symbol'].unique()[i]]['date'], y=rolling_mean_50.loc[rolling_mean_50['symbol']==rolling_mean_50['symbol'].unique()[i]]['close'],mode='lines',name=time.reset_index()['symbol'].unique()[i]))


            fig.update_layout(autosize=False,width=800,height=800,)

            st.plotly_chart(fig)

# ------------------------------ GRÁFICOS DE MÉDIAS MÓVEIS 20---------------------------- 

            rolling_50  = time['close'].rolling(window=20)
            rolling_mean_50 = rolling_50.mean()
            rolling_mean_50 = pd.DataFrame(rolling_mean_50.reset_index())
            # mm50 = time.reset_index()


            layout = go.Layout(title="MÉDIAS MÓVEIS 20",xaxis=dict(title="Data"), yaxis=dict(title="Preço R$"))
            fig = go.Figure(layout = layout)
            for i in range(len(rolling_mean_50['symbol'].unique())):
              fig.add_trace(go.Scatter(x=rolling_mean_50.loc[rolling_mean_50['symbol']==rolling_mean_50['symbol'].unique()[i]]['date'], y=rolling_mean_50.loc[rolling_mean_50['symbol']==rolling_mean_50['symbol'].unique()[i]]['close'],mode='lines',name=time.reset_index()['symbol'].unique()[i]))


            fig.update_layout(autosize=False,width=800,height=800,)

            st.plotly_chart(fig)

# ------------------------------ GRÁFICOS DE volatilidade--------------------------- 

            TRADING_DAYS = 360
            returns = np.log(time['close']/time['close'].shift(1))
            returns.fillna(0, inplace=True)
            volatility = returns.rolling(window=TRADING_DAYS).std()*np.sqrt(TRADING_DAYS)
            vol = pd.DataFrame(volatility).reset_index()
            vol = vol.dropna()

            layout = go.Layout(title=f"Volatilidade",xaxis=dict(title="Data"), yaxis=dict(title="Volatilidade"))
            fig = go.Figure(layout = layout)
            for i in range(len(vol['symbol'].unique())):
              fig.add_trace(go.Scatter(x=vol.loc[vol['symbol']==vol['symbol'].unique()[i]]['date'], y=vol.loc[vol['symbol']==vol['symbol'].unique()[i]]['close'],name=vol['symbol'].unique()[i] ))

            fig.update_layout(autosize=False,width=800,height=400,)

            st.plotly_chart(fig)

# ------------------------------ GRÁFICOS DE sharpe_ratio--------------------------- 

            sharpe_ratio = returns.mean()/volatility
            sharpe = pd.DataFrame(sharpe_ratio).reset_index()
            sharpe = sharpe.dropna()

            layout = go.Layout(title=f"SHARP (Risco / Volatilidade)",xaxis=dict(title="Data"), yaxis=dict(title="Sharp"))
            fig = go.Figure(layout = layout)
            for i in range(len(sharpe['symbol'].unique())):
              fig.add_trace(go.Scatter(x=sharpe.loc[sharpe['symbol']==sharpe['symbol'].unique()[i]]['date'], y=sharpe.loc[sharpe['symbol']==sharpe['symbol'].unique()[i]]['close'],name=sharpe['symbol'].unique()[i] ))

            fig.update_layout(autosize=False,width=800,height=400,)

            st.plotly_chart(fig)

# ------------------------------ GRÁFICOS DE correlação-------------------------- 
            st.subheader('Correlação')
            time = time.reset_index()
            time = time[['symbol','date','close']]
            df_1 = time.loc[time['symbol'] == time['symbol'].unique()[0]]
            df_1 = df_1.set_index('date')
            df_1.columns = df_1.columns.values + '-' + df_1.symbol.unique() 
            df_1.drop(df_1.columns[0],axis=1,inplace=True)
            df_2 = time.loc[time['symbol'] == time['symbol'].unique()[1]]
            df_2 = df_2.set_index('date')
            df_2.columns = df_2.columns.values + '-' + df_2.symbol.unique() 
            df_2.drop(df_2.columns[0],axis=1,inplace=True)
            df_3 = time.loc[time['symbol'] == time['symbol'].unique()[2]]
            df_3 = df_3.set_index('date')
            df_3.columns = df_3.columns.values + '-' + df_3.symbol.unique() 
            df_3.drop(df_3.columns[0],axis=1,inplace=True)
            df_4 = time.loc[time['symbol'] == time['symbol'].unique()[3]]
            df_4 = df_4.set_index('date')
            df_4.columns = df_4.columns.values + '-' + df_4.symbol.unique() 
            df_4.drop(df_4.columns[0],axis=1,inplace=True)

            merged = pd.merge(pd.merge(pd.merge(df_1,df_2,left_on=df_1.index,right_on=df_2.index,how='left'),df_3,left_on='key_0',right_on=df_3.index,how='left'),df_4,left_on='key_0',right_on=df_4.index,how='left').rename({'key_0':'date'},axis=1).set_index('date')

            retscomp = merged.pct_change()

            plt.figure(figsize=(10,8))
            sns.heatmap(retscomp.corr(),annot=True)

            st.pyplot()

# ------------------------------ GRÁFICOS DE mapa de risco-------------------------- 

            map = returns.reset_index()
            layout = go.Layout(title=f"Mapa de Risco x Retorno",xaxis=dict(title="Retorno esperado"), yaxis=dict(title="Risco"))
            fig = go.Figure(layout = layout)
            for i in range(len(map['symbol'].unique())):
              fig.add_trace(go.Scatter(x=[map.loc[map['symbol']==map['symbol'].unique()[i]]['close'].mean() * 100], y=[map.loc[map['symbol']==map['symbol'].unique()[i]]['close'].std() * 100],name=map['symbol'].unique()[i],marker=dict(size=30)))
            #fig.add_trace(go.Scatter(x=[map['close'].mean()], y=[map['close'].std()],text=map['symbol'].unique()))
            fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Red')#, range=[-0.005, 0.01])
            fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='Red')#, range=[-0.01, 0.1])
            fig.update_traces(textposition='top center')
            fig.update_layout(autosize=False,width=800,height=600,)

            st.plotly_chart(fig)

# ------------------------------ INÍCIO Comparação de ativos ------------------------------------------------------------------------------------

    if n_sprites == "Descobrir novos ativos":

        st.image('https://media.giphy.com/media/3ohs4gux2zjc7f361O/giphy.gif', width=400)    
        #image = Image.open('imagens/logo.jpg')
        #st.image(image, use_column_width=True)                       
        st.title('Descobrir novos ativos')

        PL_mínimo = int(st.number_input(label='PL_mínimo',value=10))
        PL_máximo = int(st.number_input(label='PL_máximo',value=15))
        PVP_mínimo = int(st.number_input(label='PVP_mínimo',value=0.7))
        PVP_máximo = int(st.number_input(label='PVP_máximo',value=1.5))
        DY_mínimo = int(st.number_input(label='DY_mínimo',value=4))
        DY_máximo = int(st.number_input(label='DY_máximo',value=30))

        lista = get_data()
        todos = pd.DataFrame(flatten(lista).keys()).transpose()
        todos.columns = todos.iloc[0]

        for i in range(len(lista)):
          todos = pd.concat([todos,pd.DataFrame(lista[i]).transpose()])

        todos = todos.iloc[1:]
        todos['P/L'] = todos['P/L'].str.replace('.','')
        todos['DY'] = todos['DY'].str.replace('%','')
        todos['Liq.2m.'] = todos['Liq.2m.'].str.replace('.','')
        todos['Pat.Liq'] = todos['Pat.Liq'].str.replace('.','')
        todos = todos.replace(',','.', regex=True)
        todos = todos.apply(pd.to_numeric,errors='ignore')


        if st.checkbox("Filtrar"):

            st.dataframe(todos.loc[(todos['P/L']>= PL_mínimo) & (todos['P/L']<= PL_máximo) & (todos['P/VP']>= PVP_mínimo) & (todos['P/VP']<= PVP_máximo) & (todos['DY']>= DY_mínimo) & (todos['DY']<= DY_máximo)])
            
            
            
            
# ------------------------------ INÍCIO Rastreador de trade ------------------------------------------------------------------------------------

    if n_sprites == "Rastreador de trade":

        st.image('https://media.giphy.com/media/d83YIjgW4uyTpYfjbd/giphy.gif', width=400)    
        #image = Image.open('imagens/logo.jpg')
        #st.image(image, use_column_width=True)  
        
        lista = get_data()
        todos = pd.DataFrame(flatten(lista).keys()).transpose()
        todos.columns = todos.iloc[0]
        
        for i in range(len(lista)):
          todos = pd.concat([todos,pd.DataFrame(lista[i]).transpose()])
        
        todos = todos.iloc[1:]
        
        
        start = (dt.datetime.today() + dt.timedelta(days=-300)).strftime(format='20%y-%m-%d')
        dia_limite = (dt.datetime.today() + dt.timedelta(days=-30)).strftime(format='20%y-%m-%d')
        
        st.title('Rastreador de trade')
        
        st.write('Este rastreador identifica oportunidades para swing trade vasculhando as principais ações listadas na B3, o filtro consiste em encontrar ativos que tenham médias móveis exponenciais de 9 e 72 cruzadas para cima')
        
        with st.beta_expander("Aguarde estamos vasculhando todas as ações da bolsa (Mantenha esta barra minimizada)!"):
            save = []
            #for i in range(len(tudo)):
            for i in range(len(todos)):
              try:

                #nome_do_ativo = str(tudo.iloc[i][0] + '.SA')
                nome_do_ativo = str(todos.index[i] + '.SA')
                try:
                  df = Ticker(nome_do_ativo ,country='Brazil')
                  time = df.history( start= start )
                  rolling_9  = time['close'].rolling(window=9)
                  rolling_mean_9 = rolling_9.mean().round(1)

                  rolling_72  = time['close'].rolling(window=72)
                  rolling_mean_72 = rolling_72.mean().round(1)
                  time['MM9'] = rolling_mean_9.fillna(0)
                  time['MM72'] = rolling_mean_72.fillna(0)
                  time['cruzamento'] =  time['MM9'] - time['MM72']
                  buy = time.tail(30).loc[(time.tail(30)['cruzamento']==0)]
                except:
                  exit


              except:
                exit


              if buy.empty == False:
                try:
                  if time['MM72'].iloc[-1] < time['MM9'].iloc[-1] :
                    save.append(buy.index[0][0])
                    print(buy.index[0][0])
                    #layout = go.Layout(title="Resultados",xaxis=dict(title="Data"), yaxis=dict(title="Preço R$"))
                    #fig = go.Figure(layout = layout)
                    #fig.add_trace(go.Candlestick(x=time.reset_index()['date'][-50:], open=time['open'][-50:],high=time['high'][-50:],low=time['low'][-50:],close=time['close'][-50:]))
                    #fig.update_layout(autosize=False,width=1000,height=800,)
                    #fig.show()
                    #print()
                  else:
                    continue
                except:         
                  exit

              else:
                exit
           
            
        st.dataframe(save)
        save = pd.DataFrame(save)
        
        from plotly.subplots import make_subplots
        
        
        for i in range(len(save)):
            df = Ticker(save.iloc[i] ,country='Brazil')
            time = df.history( start= start )



            fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
            vertical_spacing=0.03, subplot_titles=(st.write(save.iloc[i]), 'Volume'), 
            row_width=[0.2, 0.7])

            # Plot OHLC on 1st row
            fig.add_trace(go.Candlestick(x=time.reset_index()['date'][-90:],
                        open=time['open'][-90:], high=time['high'][-90:],
                        low=time['low'][-90:], close=time['close'][-90:], name="OHLC"), 
                        row=1, col=1)            

            # Bar trace for volumes on 2nd row without legend
            fig.add_trace(go.Bar(x=time.reset_index()['date'][-90:], y=time['volume'][-90:], showlegend=False), row=2, col=1)

            # Do not show OHLC's rangeslider plot 
            fig.update(layout_xaxis_rangeslider_visible=False)
            fig.update_layout(autosize=False,width=800,height=800,)
            st.plotly_chart(fig)
        
# ------------------------------ INÍCIO Análise de carteira ------------------------------------------------------------------------------------        
        


    if n_sprites == "Análise de carteira e previsão de lucro":
        st.image('https://media.giphy.com/media/3ShFD4IvX96027JjhH/giphy.gif', width=400)    
        #image = Image.open('imagens/logo.jpg')
        #st.image(image, use_column_width=True)                       
        st.title('Análise de carteira e previsão de lucro')
        st.subheader('Receba insights sobre suas operações realizadas no passado e preveja se sua próxima operação no futuro será lucrativa, ou não!')
        st.write('Usando os dados do seu extrato histórico fornecido pelo site da B3 iremos treinar um algorítimo de inteligência artificial que será capaz de analisar suas operações passadas, mostrar padrões que te levaram ao lucro ou prejuízo, além de prever a probabilidade de lucro de uma ação caso ela seja comprada hoje por você')
                 
        with st.beta_expander("Passo a passo de como acessar os dados no site da B3"):
            st.write('Explicar passo a passo')
            
            
        st.subheader('Faça upload aqui do seu extrato da B3')
        file  = st.file_uploader('Entre com seu extrato (.xlsx)', type = 'xlsx')    
        if file:
            df = pd.read_excel(file)

            st.dataframe(df)
            #st.table(df)
            st.write('Lucro Total até hoje: R$',round(df['Preço Médio (Venda)'].sum(),2))
            extrato = pd.DataFrame(df)

            nome_do_ativo = str(df['Código de Negociação'][0] + '.SA')
            st.subheader('Analisando os dados')
            df = Ticker(nome_do_ativo,country='Brazil')
            #time = df.history( period='max')
            time = df.history( start= extrato['Período (Inicial)'][0])
            st.dataframe(time.tail())
            #st.write(datetime.strptime(extrato['Período (Inicial)'][0], '%d/%m/%Y').strftime('%Y-%m-%d'))
            #datetimeobject = datetime.strptime(extrato['Período (Inicial)'],'%d/%m/%Y')
            #newformat = datetimeobject.strftime('%Y-%m-%d')
            #st.write(newformat)
        
# ------------------------------ FIM ----------------------------

        
if __name__ == '__main__':
    main()



In [13]:
import pandas as pd
import numpy as np
from yahooquery import Ticker
import yfinance as yf
from yahoofinancials import YahooFinancials

import datetime as dt 


In [31]:
df = pd.read_excel('../data/dadosb3.xlsx')#, index_col=0) 

In [32]:
df.head(10)

,Data do Negócio,Tipo de Movimentação,Mercado,Prazo/Vencimento,Instituição,Código de Negociação,Quantidade,Preço,Valor
0,29/07/2021,Compra,Mercado à Vista,-,ORAMA DTVM S A,BBSE3,100,21.67,2167.00
1,27/07/2021,Venda,Mercado à Vista,-,ORAMA DTVM S A,POMO4,300,3.04,912.00
2,26/07/2021,Venda,Mercado à Vista,-,ORAMA DTVM S A,CTSA3,200,2.88,576.00
3,26/07/2021,Venda,Mercado à Vista,-,ORAMA DTVM S A,MILS3,100,8.04,804.00
4,23/07/2021,Compra,Mercado à Vista,-,ORAMA DTVM S A,POMO4,300,3.17,951.00
5,14/07/2021,Compra,Mercado à Vista,-,ORAMA DTVM S A,BPAN4,100,22.58,2258.00
6,08/07/2021,Venda,Mercado Fracionário,-,ORAMA DTVM S A,GOAU3F,5,12.52,62.60
7,05/07/2021,Venda,Mercado à Vista,-,ORAMA DTVM S A,AMAR3,200,8.46,1692.00
8,05/07/2021,Venda,Mercado Fracionário,-,ORAMA DTVM S A,PETZ3F,1,23.85,23.85
9,30/06/2021,Venda,Mercado à Vista,-,ORAMA DTVM S A,BOBR4,100,2.58,258.00


In [33]:
# retirar fii
lista = []
retirar = []
for i in range(len(df['Código de Negociação'])):
    if len(df.iloc[i]['Código de Negociação']) == 5:
        lista.append(df.iloc[i]['Código de Negociação'])
    
    elif df.iloc[i]['Código de Negociação'][-1] == '1':
        retirar.append(df.iloc[i]['Código de Negociação'][-1])
    else:
        lista.append(df.iloc[i]['Código de Negociação'][:-1])

In [34]:
lista = pd.DataFrame(lista)[0].unique()

In [35]:
lista

array(['BBSE3', 'POMO4', 'CTSA3', 'MILS3', 'BPAN4', 'GOAU3', 'AMAR3',
       'PETZ3', 'BOBR4', 'FESA4', 'TAEE4', 'RANI3', 'ENEV3', 'MWET4',
       'BBDC4', 'CSMG3', 'ENGI4', 'MULT3', 'PCAR3', 'PSSA3', 'RAPT4',
       'SULA4', 'UNIP6', 'ARZZ3', 'MGEL4', 'GUAR3', 'PLAS3', 'EMBR3',
       'GGBR4', 'CSAN3', 'EQPA3', 'EQTL3', 'LEVE3', 'MRFG3', 'USIM5',
       'TUPY3', 'U1BE3', 'JPSA3', 'ENAT3', 'PETR4', 'SAPR4', 'VALE3',
       'ITUB4', 'TIET4', 'TASA4', 'ITSA4'], dtype=object)

In [36]:
lista_input = []
for i in range(len(lista)):
    
    lista_input.append(str(lista[i] + '.SA'))
    
    
    #nome_do_ativo = 'FESA4.SA'

In [37]:
lista_input

['BBSE3.SA',
 'POMO4.SA',
 'CTSA3.SA',
 'MILS3.SA',
 'BPAN4.SA',
 'GOAU3.SA',
 'AMAR3.SA',
 'PETZ3.SA',
 'BOBR4.SA',
 'FESA4.SA',
 'TAEE4.SA',
 'RANI3.SA',
 'ENEV3.SA',
 'MWET4.SA',
 'BBDC4.SA',
 'CSMG3.SA',
 'ENGI4.SA',
 'MULT3.SA',
 'PCAR3.SA',
 'PSSA3.SA',
 'RAPT4.SA',
 'SULA4.SA',
 'UNIP6.SA',
 'ARZZ3.SA',
 'MGEL4.SA',
 'GUAR3.SA',
 'PLAS3.SA',
 'EMBR3.SA',
 'GGBR4.SA',
 'CSAN3.SA',
 'EQPA3.SA',
 'EQTL3.SA',
 'LEVE3.SA',
 'MRFG3.SA',
 'USIM5.SA',
 'TUPY3.SA',
 'U1BE3.SA',
 'JPSA3.SA',
 'ENAT3.SA',
 'PETR4.SA',
 'SAPR4.SA',
 'VALE3.SA',
 'ITUB4.SA',
 'TIET4.SA',
 'TASA4.SA',
 'ITSA4.SA']

In [38]:
date_year_ago = dt.datetime.today() - dt.timedelta(days=365)
date_year_ago = date_year_ago.strftime(format='20%y-%m-%d')

In [39]:
data = yf.download(lista_input,start=date_year_ago) #'2021-1-1',)#['Adj Close']

[*********************100%***********************]  46 of 46 completed

2 Failed downloads:
- U1BE3.SA: No data found, symbol may be delisted
- TIET4.SA: No data found, symbol may be delisted


In [41]:
data

Adj Close                                                       \
            AMAR3.SA   ARZZ3.SA   BBDC4.SA   BBSE3.SA BOBR4.SA   BPAN4.SA   
Date                                                                        
2020-09-18      8.24  53.843548  17.415113  24.570147     2.01   8.293893   
2020-09-21      7.80  53.448368  17.154928  24.128651     2.01   8.058882   
2020-09-22      7.83  54.485722  17.259005  24.253422     1.99   8.127427   
2020-09-23      7.44  53.003788  16.851376  23.715948     1.95   7.843456   
2020-09-24      7.37  53.250774  17.172274  24.253422     1.96   8.039297   
...              ...        ...        ...        ...      ...        ...   
2021-09-13      6.20  84.099998  21.370001  19.570000     2.11  17.250000   
2021-09-14      6.13  84.680000  21.230000  19.620001     2.10  17.320000   
2021-09-15      5.99  84.379997  21.040001  19.219999     2.03  17.590000   
2021-09-16      5.79  83.919998  20.770000  19.400000     2.00  17.200001   
2021-09-17      5.75  83.290001  20.020000  19.330000     2.01  16.680000   

                                                      ...     Volume           \
             CSAN3.SA   CSMG3.SA CTSA3.SA   EMBR3.SA  ...   SAPR4.SA SULA4.SA   
Date                                                  ...                       
2020-09-18  18.298542  13.545430     2.50   6.680000  ...  3357200.0  14153.0   
2020-09-21  18.128035  13.476461     2.54   6.360000  ...  2710000.0  10804.0   
2020-09-22  17.784550  13.716473     2.49   6.180000  ...  2206000.0   5185.0   
2020-09-23  17.040747  13.571402     2.45   6.040000  ...  1879200.0   3349.0   
2020-09-24  17.171715  13.604841     2.45   6.180000  ...  2022500.0   3457.0   
...               ...        ...      ...        ...  ...        ...      ...   
2021-09-13  22.620001  14.510000     2.46  21.870001  ...  3718500.0  24100.0   
2021-09-14  23.500000  14.400000     2.42  21.330000  ...  3036300.0  18200.0   
2021-09-15  23.549999  14.430000     2.44  21.440001  ...  4292600.0  40700.0   
2021-09-16  23.650000  14.430000     2.42  21.250000  ...  4090100.0  44000.0   
2021-09-17  23.389999  14.570000     2.41  20.879999  ...  6264800.0  35800.0   

                                                                        \
            TAEE4.SA   TASA4.SA TIET4.SA   TUPY3.SA U1BE3.SA  UNIP6.SA   
Date                                                                     
2020-09-18  178400.0  2538800.0      NaN   694700.0      NaN  148000.0   
2020-09-21  174300.0  1789600.0      NaN  1352900.0      NaN  113700.0   
2020-09-22   88800.0  2244600.0      NaN   726500.0      NaN   95500.0   
2020-09-23   89400.0  2367600.0      NaN   755300.0      NaN   95800.0   
2020-09-24   72700.0  2104100.0      NaN   731000.0      NaN   81700.0   
...              ...        ...      ...        ...      ...       ...   
2021-09-13  192100.0  1404700.0      NaN   728200.0      NaN  251500.0   
2021-09-14  130200.0  1366000.0      NaN   661200.0      NaN  175700.0   
2021-09-15  121200.0  2528800.0      NaN   783300.0      NaN  270500.0   
2021-09-16   87800.0  1392100.0      NaN  1083500.0      NaN  338300.0   
2021-09-17  255500.0  1473300.0      NaN  1370700.0      NaN  537200.0   

                                    
              USIM5.SA    VALE3.SA  
Date                                
2020-09-18  37647500.0  33143800.0  
2020-09-21  21907600.0  53120800.0  
2020-09-22  19147000.0  23996200.0  
2020-09-23  20167200.0  37365600.0  
2020-09-24  20479400.0  31607500.0  
...                ...         ...  
2021-09-13  22413100.0  18832800.0  
2021-09-14  20735600.0  25571600.0  
2021-09-15  40652600.0  35084400.0  
2021-09-16  21565500.0  39726500.0  
2021-09-17  31966800.0  74648800.0  

[247 rows x 276 columns]

In [229]:
df_filled = pd.DataFrame(columns = ['name'])
df_filled['name'] = lista_input

# Preencher o dataset com as infos para o modelo, lucro ou preju, indicadores partir da data da compra...

In [154]:
df_filled

,name,situation,Preço_médio_comprado,Preço_médio_vendido,Ganho_total,Rendimento_total,Rendimento_total_%
0,BBSE3.SA,Comprada,0.000,0.000,0.000,0,NaN
1,POMO4.SA,Vendida,3.170,3.040,-39.000,0,-4.28
2,CTSA3.SA,Vendida,3.400,2.880,-104.000,0,-18.06
3,MILS3.SA,Vendida,8.100,8.040,-6.000,0,-0.75
4,BPAN4.SA,Comprada + histórico,21.140,25.610,22.350,0,17.45
5,GOAU3.SA,2 históricos,12.150,11.430,-4.320,0,-6.30
6,AMAR3.SA,Vendida,9.210,8.460,-150.000,0,-8.87
7,PETZ3.SA,Vendida,24.860,23.850,-1.010,0,-4.23
8,BOBR4.SA,Vendida,2.790,2.580,-21.000,0,-8.14
9,FESA4.SA,Comprada + histórico,31.380,37.435,12.110,0,16.17


In [230]:
df_filled['situation'] = 'Nenhum'
for i in range(len(lista)):
    # primeiro filtro olha no data frame da B3 e segundo no dataframe criado
    if len(df.loc[df['Código de Negociação'].str.contains(lista[i])]) == 1:        
        print('1')
        df_filled.loc[df_filled['name'].str.contains(lista[i]),'situation'] = 'Comprada'
        print(lista[i])
        #adicionar lógica para garantir que não são 2 compras ou 3 compras ...
    if len(df.loc[df['Código de Negociação'].str.contains(lista[i])]) == 2: 
        print('2')
        df_filled.loc[df_filled['name'].str.contains(lista[i]),'situation'] = 'Vendida'
        print(lista[i])  
    if len(df.loc[df['Código de Negociação'].str.contains(lista[i])]) == 3: 
        print('3')
        df_filled.loc[df_filled['name'].str.contains(lista[i]),'situation'] = 'Comprada + histórico'
        print(lista[i])  
    if len(df.loc[df['Código de Negociação'].str.contains(lista[i])]) >= 4: 
        print('4')
        df_filled.loc[df_filled['name'].str.contains(lista[i]),'situation'] = '2 históricos'
        print(lista[i])  

1
BBSE3
2
POMO4
2
CTSA3
2
MILS3
3
BPAN4
4
GOAU3
2
AMAR3
2
PETZ3
2
BOBR4
3
FESA4
4
TAEE4
2
RANI3
2
ENEV3
4
MWET4
2
BBDC4
4
CSMG3
2
ENGI4
2
MULT3
2
PCAR3
2
PSSA3
2
RAPT4
2
SULA4
2
UNIP6
1
ARZZ3
2
MGEL4
1
GUAR3
1
PLAS3
3
EMBR3
3
GGBR4
1
CSAN3
3
EQPA3
2
EQTL3
1
LEVE3
1
MRFG3
3
USIM5
1
TUPY3
1
U1BE3
1
JPSA3
2
ENAT3
4
PETR4
3
SAPR4
2
VALE3
2
ITUB4
2
TIET4
2
TASA4
2
ITSA4


# Tentando aidcionar parte com try, fora do for está funcionando como exemplo abaixo, mas no for não vai, investigar, tentar colcoa rcresimento semana, mes e 3 meses

In [244]:
# lógica para saber se houve compra e venda ou somente compra, apartir daqui o pressuposto de que houve o trade, se não só hold

#df_filled['Preço_médio_comprado'] = 0
#df_filled['Preço_médio_vendido'] = 0
#df_filled['Ganho_total'] = 0
#df_filled['Rendimento_total_%'] = 0

for i in range(len(lista)):
    if df.loc[(df['Código de Negociação'].str.contains(lista[i]))].sort_values('Tipo de Movimentação')[-1:]['Tipo de Movimentação'].item() == 'Venda':
        
        #Construção das variáveis
        preco_medio_compra = df.loc[(df['Código de Negociação'].str.contains(lista[i])) & (df['Tipo de Movimentação'] == 'Compra' )]['Preço'].mean()
        quantidade_comprada = df.loc[(df['Código de Negociação'].str.contains(lista[i])) & (df['Tipo de Movimentação'] == 'Compra' )]['Quantidade'].sum()
        preco_medio_vendido = df.loc[(df['Código de Negociação'].str.contains(lista[i])) & (df['Tipo de Movimentação'] == 'Venda' )]['Preço'].mean()
        quantidade_vendida = df.loc[(df['Código de Negociação'].str.contains(lista[i])) & (df['Tipo de Movimentação'] == 'Venda' )]['Quantidade'].sum()
        
        data_compra_1 = df.loc[(df['Código de Negociação'].str.contains(lista[i]))].sort_values('Data do Negócio')[-1:]['Data do Negócio'].item()
        Ganho_total = (preco_medio_vendido - preco_medio_compra) * quantidade_vendida
        rendimento_total = round(((preco_medio_vendido - preco_medio_compra) / preco_medio_vendido) * 100,2)
        
        #dados históricos ticker
        try:
            dados_acao = pd.DataFrame(data.loc[ : , (['Open','High','Low','Close','Adj Close','Volume'],lista[i])])
            dados_acao_filtrado = dados_acao.loc[dados_acao.index <= data_compra_1]

            #valor da ultima cotação
            cotacao_last = dados_acao_filtrado['Close'][-1:][lista[i]][0]
            #valor cotação 7 dias atras
            cotacao_7 = dados_acao_filtrado['Close'][-7:-6][lista[i]][0]
            #% da queda ou aumento ultimos 7 dias
            crescimento_7 = round(((cotacao_last - cotacao_7) / cotacao_last) * 100,2)
            
            df_filled.loc[df_filled['name'].str.contains(lista[i]),'Crescimento_ultimos_7_dias'] = crescimento_7
        except:
            exit
        
        
        
        #Atribuições
        df_filled.loc[df_filled['name'].str.contains(lista[i]),'data_compra_1'] = data_compra_1
        df_filled.loc[df_filled['name'].str.contains(lista[i]),'Preço_médio_comprado'] = preco_medio_compra
        df_filled.loc[df_filled['name'].str.contains(lista[i]),'Preço_médio_vendido'] = preco_medio_vendido
        df_filled.loc[df_filled['name'].str.contains(lista[i]),'Ganho_total'] = Ganho_total
        df_filled.loc[df_filled['name'].str.contains(lista[i]),'Rendimento_total_%'] = rendimento_total
        
        
        
        
    # casos em que ainda não vendeu    
    if df.loc[(df['Código de Negociação'].str.contains(lista[i]))].sort_values('Tipo de Movimentação')[-1:]['Tipo de Movimentação'].item() == 'Compra':
        print(lista[i])

BBSE3


# testes

In [239]:
dados_acao

,Adj Close,Close,High,Low,Open,Volume
,POMO4.SA,POMO4.SA,POMO4.SA,POMO4.SA,POMO4.SA,POMO4.SA
Date,,,,,,
2020-09-18,2.807800,2.91,2.97,2.89,2.96,5340800.0
2020-09-21,2.682365,2.78,2.88,2.77,2.88,9828400.0
2020-09-22,2.696970,2.77,2.81,2.72,2.79,8563500.0
2020-09-23,2.609343,2.68,2.82,2.68,2.78,8748100.0
2020-09-24,2.648288,2.72,2.77,2.66,2.70,6085000.0
...,...,...,...,...,...,...
2021-09-13,2.740000,2.74,2.76,2.65,2.69,3656800.0
2021-09-14,2.690000,2.69,2.81,2.68,2.74,2967600.0


In [240]:
dados_acao_filtrado

,Adj Close,Close,High,Low,Open,Volume
,POMO4.SA,POMO4.SA,POMO4.SA,POMO4.SA,POMO4.SA,POMO4.SA
Date,,,,,,
2020-09-18,2.807800,2.91,2.97,2.89,2.96,5340800.0
2020-09-21,2.682365,2.78,2.88,2.77,2.88,9828400.0
2020-09-22,2.696970,2.77,2.81,2.72,2.79,8563500.0
2020-09-23,2.609343,2.68,2.82,2.68,2.78,8748100.0
2020-09-24,2.648288,2.72,2.77,2.66,2.70,6085000.0
...,...,...,...,...,...,...
2021-07-21,3.140000,3.14,3.25,3.14,3.22,5406500.0
2021-07-22,3.160000,3.16,3.19,3.12,3.13,2829200.0


In [173]:
df.loc[(df['Código de Negociação'].str.contains('CSMG3F'))].sort_values('Data do Negócio')[-1:]['Data do Negócio'].item()

'29/04/2021'

In [170]:
df.loc[(df['Código de Negociação'].str.contains('CSMG3F'))]

,Data do Negócio,Tipo de Movimentação,Mercado,Prazo/Vencimento,Instituição,Código de Negociação,Quantidade,Preço,Valor
29,01/06/2021,Venda,Mercado Fracionário,-,ORAMA DTVM S A,CSMG3F,1,17.56,17.56
67,29/04/2021,Compra,Mercado Fracionário,-,ORAMA DTVM S A,CSMG3F,1,16.74,16.74
86,23/03/2021,Venda,Mercado Fracionário,-,CLEAR CORRETORA - GRUPO XP,CSMG3F,1,15.25,15.25
92,17/03/2021,Compra,Mercado Fracionário,-,CLEAR CORRETORA - GRUPO XP,CSMG3F,1,14.55,14.55


In [238]:
dados_acao = pd.DataFrame(data.loc[ : , (['Open','High','Low','Close','Adj Close','Volume'],['POMO4.SA'])])
dados_acao_filtrado = dados_acao.loc[dados_acao.index <= '27/07/2021']

#valor da ultima cotação
cotacao_last = dados_acao_filtrado['Close'][-1:]['POMO4.SA'][0]
#valor cotação 7 dias atras
cotacao_7 = dados_acao_filtrado['Close'][-7:-6]['POMO4.SA'][0]
#% da queda ou aumento ultimos 7 dias
round(((cotacao_last - cotacao_7) / cotacao_last) * 100,2)

-1.31

In [236]:
data.loc[ : , (['Open','High','Low','Close','Adj Close','Volume'],['POMO4.SA'])]

,Adj Close,Close,High,Low,Open,Volume
,POMO4.SA,POMO4.SA,POMO4.SA,POMO4.SA,POMO4.SA,POMO4.SA
Date,,,,,,
2020-09-18,2.807800,2.91,2.97,2.89,2.96,5340800.0
2020-09-21,2.682365,2.78,2.88,2.77,2.88,9828400.0
2020-09-22,2.696970,2.77,2.81,2.72,2.79,8563500.0
2020-09-23,2.609343,2.68,2.82,2.68,2.78,8748100.0
2020-09-24,2.648288,2.72,2.77,2.66,2.70,6085000.0
...,...,...,...,...,...,...
2021-09-13,2.740000,2.74,2.76,2.65,2.69,3656800.0
2021-09-14,2.690000,2.69,2.81,2.68,2.74,2967600.0


In [168]:
df

,Data do Negócio,Tipo de Movimentação,Mercado,Prazo/Vencimento,Instituição,Código de Negociação,Quantidade,Preço,Valor
0,29/07/2021,Compra,Mercado à Vista,-,ORAMA DTVM S A,BBSE3,100,21.67,2167.00
1,27/07/2021,Venda,Mercado à Vista,-,ORAMA DTVM S A,POMO4,300,3.04,912.00
2,26/07/2021,Venda,Mercado à Vista,-,ORAMA DTVM S A,CTSA3,200,2.88,576.00
3,26/07/2021,Venda,Mercado à Vista,-,ORAMA DTVM S A,MILS3,100,8.04,804.00
4,23/07/2021,Compra,Mercado à Vista,-,ORAMA DTVM S A,POMO4,300,3.17,951.00
...,...,...,...,...,...,...,...,...,...
106,10/03/2021,Venda,Mercado à Vista,-,CLEAR CORRETORA - GRUPO XP,ITSA4,100,10.20,1020.00
107,05/03/2021,Compra,Mercado à Vista,-,CLEAR CORRETORA - GRUPO XP,ITSA4,100,10.40,1040.00
108,05/03/2021,Compra,Mercado à Vista,-,CLEAR CORRETORA - GRUPO XP,PETR4,100,23.00,2300.00
109,25/02/2021,Compra,Mercado Fracionário,-,CLEAR CORRETORA - GRUPO XP,PETR4F,1,24.35,24.35


In [245]:
df_filled

,name,situation,data_compra_1,Preço_médio_comprado,Preço_médio_vendido,Ganho_total,Rendimento_total_%
0,BBSE3.SA,Comprada,NaN,NaN,NaN,NaN,NaN
1,POMO4.SA,Vendida,27/07/2021,3.170,3.040,-39.000,-4.28
2,CTSA3.SA,Vendida,26/07/2021,3.400,2.880,-104.000,-18.06
3,MILS3.SA,Vendida,26/07/2021,8.100,8.040,-6.000,-0.75
4,BPAN4.SA,Comprada + histórico,20/05/2021,21.140,25.610,22.350,17.45
5,GOAU3.SA,2 históricos,24/03/2021,12.150,11.430,-4.320,-6.30
6,AMAR3.SA,Vendida,16/06/2021,9.210,8.460,-150.000,-8.87
7,PETZ3.SA,Vendida,07/05/2021,24.860,23.850,-1.010,-4.23
8,BOBR4.SA,Vendida,30/06/2021,2.790,2.580,-21.000,-8.14
9,FESA4.SA,Comprada + histórico,25/06/2021,31.380,37.435,12.110,16.17


In [166]:
df_filled['Ganho_total'].sum()

671.1699999999998

# ÁREA DE TESTES  \/

In [ ]:
#Média móvel
rolling_20  = dados_acao_filtrado['Close'].rolling(window=20)
rolling_mean_20 = rolling_20.mean()

# Lógica para avaliar lucro ou prejuizo nas diversas opssibilidades de cenário

In [161]:
# em andamento, descarta ou tenta avaliar se é um bom momento para venda
df.loc[df['Código de Negociação'].str.contains(lista[0])].sort_index()

,Data do Negócio,Tipo de Movimentação,Mercado,Prazo/Vencimento,Instituição,Código de Negociação,Quantidade,Preço,Valor
0,29/07/2021,Compra,Mercado à Vista,-,ORAMA DTVM S A,BBSE3,100,21.67,2167.0


In [160]:
# comprou e vendeu
df.loc[df['Código de Negociação'].str.contains(lista[1])].sort_index()

,Data do Negócio,Tipo de Movimentação,Mercado,Prazo/Vencimento,Instituição,Código de Negociação,Quantidade,Preço,Valor
1,27/07/2021,Venda,Mercado à Vista,-,ORAMA DTVM S A,POMO4,300,3.04,912.0
4,23/07/2021,Compra,Mercado à Vista,-,ORAMA DTVM S A,POMO4,300,3.17,951.0


In [163]:
# comprou e vendeu e comprou de novo ( avalia última rodada de compra e venda)
df.loc[df['Código de Negociação'].str.contains(lista[4])].sort_index()

,Data do Negócio,Tipo de Movimentação,Mercado,Prazo/Vencimento,Instituição,Código de Negociação,Quantidade,Preço,Valor
5,14/07/2021,Compra,Mercado à Vista,-,ORAMA DTVM S A,BPAN4,100,22.58,2258.00
21,16/06/2021,Venda,Mercado Fracionário,-,ORAMA DTVM S A,BPAN4F,5,25.61,128.05
42,20/05/2021,Compra,Mercado Fracionário,-,ORAMA DTVM S A,BPAN4F,5,19.70,98.50


In [159]:
# comprou e vendeu + de 1 vez
df.loc[df['Código de Negociação'].str.contains(lista[15])].sort_index()

,Data do Negócio,Tipo de Movimentação,Mercado,Prazo/Vencimento,Instituição,Código de Negociação,Quantidade,Preço,Valor
29,01/06/2021,Venda,Mercado Fracionário,-,ORAMA DTVM S A,CSMG3F,1,17.56,17.56
67,29/04/2021,Compra,Mercado Fracionário,-,ORAMA DTVM S A,CSMG3F,1,16.74,16.74
86,23/03/2021,Venda,Mercado Fracionário,-,CLEAR CORRETORA - GRUPO XP,CSMG3F,1,15.25,15.25
92,17/03/2021,Compra,Mercado Fracionário,-,CLEAR CORRETORA - GRUPO XP,CSMG3F,1,14.55,14.55


# Lógica para todos que tiveram venda

In [117]:
# lógica para saber se houve compra e venda ou somente compra, apartir daqui o pressuposto de que houve o trade, se não só hold
if df.loc[(df['Código de Negociação'].str.contains('GOAU3'))].sort_values('Tipo de Movimentação')[-1:]['Tipo de Movimentação'].item() == 'Venda':
    preco_medio_compra = df.loc[(df['Código de Negociação'].str.contains('GOAU3')) & (df['Tipo de Movimentação'] == 'Compra' )]['Preço'].mean()
    quantidade_comprada = df.loc[(df['Código de Negociação'].str.contains('GOAU3')) & (df['Tipo de Movimentação'] == 'Compra' )]['Quantidade'].sum()
    preco_medio_vendido = df.loc[(df['Código de Negociação'].str.contains('GOAU3')) & (df['Tipo de Movimentação'] == 'Venda' )]['Preço'].mean()
    quantidade_vendida = df.loc[(df['Código de Negociação'].str.contains('GOAU3')) & (df['Tipo de Movimentação'] == 'Venda' )]['Quantidade'].sum()
    
    lucro_preju = preco_medio_vendido - preco_medio_compra
    valor_ganho_perdido_operacao = (preco_medio_vendido * quantidade_vendida) - (preco_medio_compra * quantidade_comprada)

i


In [163]:
df.loc[(df['Código de Negociação'].str.contains('BBSE3'))]

,Data do Negócio,Tipo de Movimentação,Mercado,Prazo/Vencimento,Instituição,Código de Negociação,Quantidade,Preço,Valor
0,29/07/2021,Compra,Mercado à Vista,-,ORAMA DTVM S A,BBSE3,100,21.67,2167.0


In [192]:
df.loc[(df['Código de Negociação'].str.contains('GOAU3')) & (df['Tipo de Movimentação'] == 'Venda' )]

,Data do Negócio,Tipo de Movimentação,Mercado,Prazo/Vencimento,Instituição,Código de Negociação,Quantidade,Preço,Valor
6,08/07/2021,Venda,Mercado Fracionário,-,ORAMA DTVM S A,GOAU3F,5,12.52,62.60
77,24/03/2021,Venda,Mercado Fracionário,-,CLEAR CORRETORA - GRUPO XP,GOAU3F,1,10.34,10.34


In [197]:
preco_medio_compra = df.loc[(df['Código de Negociação'].str.contains('GOAU3')) & (df['Tipo de Movimentação'] == 'Compra' )]['Preço'].mean()
quantidade_comprada = df.loc[(df['Código de Negociação'].str.contains('GOAU3')) & (df['Tipo de Movimentação'] == 'Compra' )]['Quantidade'].sum()
preco_medio_vendido = df.loc[(df['Código de Negociação'].str.contains('GOAU3')) & (df['Tipo de Movimentação'] == 'Venda' )]['Preço'].mean()
quantidade_vendida = df.loc[(df['Código de Negociação'].str.contains('GOAU3')) & (df['Tipo de Movimentação'] == 'Venda' )]['Quantidade'].sum()

In [198]:
# preju
preco_medio_vendido - preco_medio_compra

-0.7200000000000006

In [199]:
# valor total da operação
(preco_medio_vendido * quantidade_vendida) - (preco_medio_compra * quantidade_comprada)

-4.320000000000007

# Aplicando lógica apenas em quem foi vendido para encher o dataset com dados

In [120]:
# lógica para saber se houve compra e venda ou somente compra, apartir daqui o pressuposto de que houve o trade, se não só hold

if df.loc[(df['Código de Negociação'].str.contains('GOAU3'))].sort_values('Tipo de Movimentação')[-1:]['Tipo de Movimentação'].item() == 'Venda':
    preco_medio_compra = df.loc[(df['Código de Negociação'].str.contains('GOAU3')) & (df['Tipo de Movimentação'] == 'Compra' )]['Preço'].mean()
    quantidade_comprada = df.loc[(df['Código de Negociação'].str.contains('GOAU3')) & (df['Tipo de Movimentação'] == 'Compra' )]['Quantidade'].sum()
    preco_medio_vendido = df.loc[(df['Código de Negociação'].str.contains('GOAU3')) & (df['Tipo de Movimentação'] == 'Venda' )]['Preço'].mean()
    quantidade_vendida = df.loc[(df['Código de Negociação'].str.contains('GOAU3')) & (df['Tipo de Movimentação'] == 'Venda' )]['Quantidade'].sum()

    lucro_preju = preco_medio_vendido - preco_medio_compra
    valor_ganho_perdido_operacao = (preco_medio_vendido * quantidade_vendida) - (preco_medio_compra * quantidade_comprada)

In [141]:
df

,Data do Negócio,Tipo de Movimentação,Mercado,Prazo/Vencimento,Instituição,Código de Negociação,Quantidade,Preço,Valor
0,29/07/2021,Compra,Mercado à Vista,-,ORAMA DTVM S A,BBSE3,100,21.67,2167.00
1,27/07/2021,Venda,Mercado à Vista,-,ORAMA DTVM S A,POMO4,300,3.04,912.00
2,26/07/2021,Venda,Mercado à Vista,-,ORAMA DTVM S A,CTSA3,200,2.88,576.00
3,26/07/2021,Venda,Mercado à Vista,-,ORAMA DTVM S A,MILS3,100,8.04,804.00
4,23/07/2021,Compra,Mercado à Vista,-,ORAMA DTVM S A,POMO4,300,3.17,951.00
...,...,...,...,...,...,...,...,...,...
106,10/03/2021,Venda,Mercado à Vista,-,CLEAR CORRETORA - GRUPO XP,ITSA4,100,10.20,1020.00
107,05/03/2021,Compra,Mercado à Vista,-,CLEAR CORRETORA - GRUPO XP,ITSA4,100,10.40,1040.00
108,05/03/2021,Compra,Mercado à Vista,-,CLEAR CORRETORA - GRUPO XP,PETR4,100,23.00,2300.00
109,25/02/2021,Compra,Mercado Fracionário,-,CLEAR CORRETORA - GRUPO XP,PETR4F,1,24.35,24.35


In [77]:
data.loc[ : , (['Open','High','Low','Close','Adj Close','Volume'],['AMAR3.SA'])]


,Adj Close,Close,High,Low,Open,Volume
,AMAR3.SA,AMAR3.SA,AMAR3.SA,AMAR3.SA,AMAR3.SA,AMAR3.SA
Date,,,,,,
2021-01-04,6.59,6.59,6.86,6.53,6.86,4333800.0
2021-01-05,6.57,6.57,6.63,6.40,6.61,3453900.0
2021-01-06,6.20,6.20,6.58,6.15,6.57,8920800.0
2021-01-07,6.11,6.11,6.27,6.02,6.21,6136400.0
2021-01-08,6.26,6.26,6.35,6.10,6.14,6049700.0
...,...,...,...,...,...,...
2021-08-30,6.59,6.59,6.80,6.57,6.71,3905900.0
2021-08-31,6.65,6.65,6.70,6.52,6.60,4089400.0


# Teste de filtro do rsatreador

In [15]:
start = (dt.datetime.today() + dt.timedelta(days=-300)).strftime(format='20%y-%m-%d')
dia_limite = (dt.datetime.today() + dt.timedelta(days=-30)).strftime(format='20%y-%m-%d')

In [30]:
#nome_do_ativo = str(tudo.iloc[i][0] + '.SA')
nome_do_ativo = str('CGRA4' + '.SA')

df = Ticker(nome_do_ativo ,country='Brazil')
time = df.history( start= start )
rolling_9  = time['close'].rolling(window=9)
rolling_mean_9 = rolling_9.mean().round(1)

rolling_72  = time['close'].rolling(window=72)
rolling_mean_72 = rolling_72.mean().round(1)
time['MM9'] = rolling_mean_9.fillna(0)
time['MM72'] = rolling_mean_72.fillna(0)
time['cruzamento'] =  time['MM9'] - time['MM72']
buy = time.tail(50).loc[(time.tail(50)['cruzamento']==0)]



if buy.empty == False:

        
        
    if time['MM72'].iloc[-1] < time['MM9'].iloc[-1] and  time.tail(1)['volume'][0] > 5000:
        #save.append(buy.index[0][0])
        print(buy.index[0][0])
        #layout = go.Layout(title="Resultados",xaxis=dict(title="Data"), yaxis=dict(title="Preço R$"))
        #fig = go.Figure(layout = layout)
        #fig.add_trace(go.Candlestick(x=time.reset_index()['date'][-50:], open=time['open'][-50:],high=time['high'][-50:],low=time['low'][-50:],close=time['close'][-50:]))
        #fig.update_layout(autosize=False,width=1000,height=800,)
        #fig.show()
        #print()


CGRA4.SA


In [18]:
buy

close  open   low      volume  high  adjclose  dividends  \
symbol   date                                                                   
POMO4.SA 2021-08-05   2.83  3.00  2.83  10138800.0  3.00      2.83        0.0   
         2021-08-06   2.75  2.81  2.75   9499700.0  2.87      2.75        0.0   

                     MM9  MM72  cruzamento  
symbol   date                               
POMO4.SA 2021-08-05  3.0   3.0         0.0  
         2021-08-06  3.0   3.0         0.0

In [28]:
time.tail(1)['volume'][0]

0.0